In [ ]:
import os

# Mocking data for ingestion job of operators

For siplicity the code is written for only two countries: DK and NL

In [ ]:
country_codes = {'DK', 'NL'}

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("NameMatching_Notebook")
    .config('spark.dynamicAllocation.enabled', False)
    .config('spark.executorEnv.PYTHON_EGG_CACHE', '/tmp')
    .config('spark.executor.instances', 4)
    .config('spark.executor.cores', 13)
    .config('spark.executor.memory', '14g')
    .config('spark.driver.memory', '7g')
    .getOrCreate()
)
sc = spark.sparkContext

In [ ]:
import pyspark.sql.functions as sf

## Mock current operator data

This will consist of only 5% of the current data.

In [ ]:
operators_old_dir = 'adl://ulohubdldevne.azuredatalakestore.net/data/parquet/'
test_output_dir = 'adl://ulohubdldevne.azuredatalakestore.net/data/parquet/test/'

oprs_old = (spark
.read.parquet(operators_old_dir + 'OPERATORS_MERGED.parquet')
.sample(False, 0.05)
.filter(sf.col('countryCode').isin(country_codes)))
            
(oprs_old
 .write
.partitionBy('countryCode')
.parquet(test_output_dir + 'OPERATORS_MERGED.parquet', mode='overwrite'))

In [ ]:
spark.read.parquet(test_output_dir + 'OPERATORS_MERGED.parquet').groupby('countryCode').count().show()

In [ ]:
oprs_old = oprs_old.withColumn('id', sf.explode(sf.col('refIds')))

## Mock input data as 1% from original

In [ ]:
# read operators and assign id
operators_matched_dir = 'adl://ulohubdldevne.azuredatalakestore.net/data/parquet/'
test_output_dir = 'adl://ulohubdldevne.azuredatalakestore.net/data/parquet/test/'

input_oprs_raw = (spark
.read.parquet(operators_matched_dir + 'OPERATORS.parquet')
.sample(False, 0.01)
.filter(sf.col('COUNTRY_CODE').isin(country_codes)))

Check for overlap and alter some records

In [ ]:
input_oprs = (
    input_oprs_raw
    .withColumn('id', sf.concat_ws('~',
                                   sf.col('COUNTRY_CODE'),
                                   sf.col('SOURCE'),
                                   sf.col('REF_OPERATOR_ID')))
)

In [ ]:
df = input_oprs.join(oprs_old, on='id', how='left')
df.persist()

print('Known Operators:', oprs_old.count())
print('Input Operators:', df.count())
print('Input Known:', df.filter(sf.col('ohubOperatorId').isNotNull()).count())
print('Input New:', df.filter(sf.col('ohubOperatorId').isNull()).count())

10% of the already known operators will be altered on the `STREET_CLEANSED`

In [ ]:
df_altered = df.withColumn('STREET_CLEANSED', sf.when(sf.col('ohubOperatorId').isNotNull() & (sf.rand(seed=0) < 0.1), 'street_changed').otherwise(sf.col('STREET_CLEANSED')))
df_altered.persist();

In [ ]:
print('Known Operators:', oprs_old.count())
print('Input Operators:', df_altered.count())
print('Input Known:', df_altered.filter(sf.col('ohubOperatorId').isNotNull()).count())
print('Input Known Equal:', df_altered.filter(sf.col('ohubOperatorId').isNotNull() &
                                              (sf.col('STREET_CLEANSED').isNull() | (sf.col('STREET_CLEANSED') != 'street_changed'))).count())
print('Input Known Changed:', df_altered.filter(sf.col('ohubOperatorId').isNotNull() & (sf.col('STREET_CLEANSED') == 'street_changed')).count())
print('Input New:', df_altered.filter(sf.col('ohubOperatorId').isNull()).count())

In [ ]:
(df_altered.select(input_oprs_raw.columns)
 .write
 .partitionBy('COUNTRY_CODE')
 .parquet(test_output_dir + 'OPERATORS.parquet', mode='overwrite'))

In [ ]:
spark.read.parquet(test_output_dir + 'OPERATORS.parquet').groupby('COUNTRY_CODE').count().show()

In [ ]:
array = np.random.randint(0, 1E5, size=10000, dtype='int')
%timeit max(array)
%timeit np.max(array)
%timeit array.max()

In [ ]:
bool_array = np.ones(10000, dtype=np.bool)
%timeit any(bool_array)
%timeit np.any(bool_array, keepdims=False)
%timeit bool_array.any(keepdims=False)

In [ ]:
bool_array = np.zeros(10000, dtype=np.bool)
%timeit any(bool_array)
%timeit np.any(bool_array, keepdims=False)
%timeit bool_array.any(keepdims=False)